In [66]:
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
#from astropy.cosmology import Planck15
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [78]:
t = Table.read('paperII_table1.fits', format='fits')

print(t)
GalRa = t['RAJ2000'].tolist()
GalDec = t['DEJ2000'].tolist() 
GalName = t['Name'].tolist()
GalZ = t['z'].tolist()
Galf_z = t['f_z'].tolist()
NewGalRa = []
NewGalDec = []

#print(GalRa)

for i in range(len(GalRa)):
    
    x = list(map(int, GalRa[i].split()))
    y = list(map(int, GalDec[i].split()))
    
    xDeg = (x[0]*15) + (x[1]*0.25) + (x[2]*0.004166)
    
    if x[0] < 0:
        yDeg = -(y[2]/3600) - (y[1]/60) + y[0]
    else:
        yDeg = -(y[2]/3600) - (y[1]/60) + y[0]
    
    NewGalRa.append(xDeg)
    NewGalDec.append(yDeg)



#print(GalRa)
#print(GalDec)

recno      Name     RAJ2000   DEJ2000   pCNN ... rAper iAper   z    f_z  Ref  
                    "h:m:s"   "d:m:s"        ...  mag   mag                   
----- ------------- -------- --------- ----- ... ----- ----- ------ --- ------
    1 PS1J2226+0041 22 26 09 +00 41 42  0.99 ...  21.9 20.63 0.6471   * a,b,c
    2 PS1J1821+7130 18 21 40 +71 30 10   1.0 ... 19.04 18.62    0.0          
    3 PS1J1647+1117 16 47 04 +11 17 49 0.951 ... 19.92 19.32    0.0          
    4 PS1J1559+3147 15 59 23 +31 47 12   1.0 ... 18.74 18.15 0.1489   *      
    5 PS1J1508-1652 15 08 10 -16 52 38   1.0 ... 20.65 19.88    0.0          
    6 PS1J1421-0536 14 21 28 -05 36 51   1.0 ... 19.99 19.29    0.0          
    7 PS1J1415+1112 14 15 31 +11 12 08 0.993 ... 19.81 19.16 0.3155   *      
    8 PS1J1322-0501 13 22 27 -05 01 34 0.995 ... 21.55 20.66    0.0          
    9 PS1J1113+0104 11 13 57 +01 04 05 0.951 ... 21.81  20.7 0.6402   *      
   10 PS1J0919+0336 09 19 05 +03 36 39 0.919 ... 20.68  19.9 

In [68]:
df = pd.read_csv('FRBDataWithRedshifts.csv')

FRBra = df['ra'].tolist()
FRBdec = df['dec'].tolist()
FRBname = df['tns_name'].tolist()
FRBz = df['redshift'].tolist()
FRBbdm = df['bonsai_dm'].tolist()
FRBdmfit = df['dm_fitb'].tolist()
FRBdmfit_er = df['dm_fitb_err'].tolist()
FRBbsnr = df['bonsai_snr'].tolist()

In [69]:
x = list(map(int, GalDec[7].split()))

print(x)

[-5, 1, 34]


In [70]:
def RaDec_to_Cartesian_FRB (Ra, Dec): 
    
    cord = SkyCoord(ra=Ra*u.degree, dec=Dec*u.degree, frame='icrs')
    
    raHour = cord.ra.hms[0]
    raMin = cord.ra.hms[1]
    raSec = cord.ra.hms[2]
    decMin = cord.dec.hms[1]
    decSec = cord.dec.hms[2]
    
    #decRadians = Dec * (np.pi/180)
    
    a = (raHour * 15) + (raMin * 0.25) + (raSec * 0.004166)
    
    if Dec < 0: 
        b = (np.absolute(Dec)) + (decMin/60) + (decSec/3600) * -(Dec) #needs negative sign
    else:
        b = (np.absolute(Dec)) + (decMin/60) + (decSec/3600) * (Dec)
        
    c = 1 #we don't care about distance?
    
    #-----------------------------------------------------------------------------------------
    
    ar = a * (np.pi/180) #in radians for numpy.sin or cos
    br = b * (np.pi/180)
    
    x = (c * np.cos(br)) * np.cos(ar)
    y = (c * np.cos(br)) * np.sin(ar)
    z = c * np.sin(br)
    
    return x, y, z

In [71]:
TransformedFRBx = []
TransformedFRBy = []
TransformedFRBz = []

for i in range(len(FRBra)):
    
    d = RaDec_to_Cartesian_FRB(FRBra[i], FRBdec[i])
    
    TransformedFRBx.append(d[0])
    TransformedFRBy.append(d[1])
    TransformedFRBz.append(d[2])
    

In [72]:
def RaDec_to_Cartesian_GAL (raHour, raMin, raSec, decDeg, decMin, decSec): 
    
    #decRadians = Dec * (np.pi/180)
    
    a = (raHour * 15) + (raMin * 0.25) + (raSec * 0.004166)
    
    if decDeg < 0: 
        b = (np.absolute(decDeg)) + (decMin/60) + (decSec/3600) * -(decDeg) #needs negative sign
    else:
        b = (np.absolute(decDeg)) + (decMin/60) + (decSec/3600) * (decDeg)
        
    c = 1 #we don't care about distance?
    
    #-----------------------------------------------------------------------------------------
    
    ar = a * (np.pi/180) #in radians for numpy.sin or cos
    br = b * (np.pi/180)
    
    x = (c * np.cos(br)) * np.cos(ar)
    y = (c * np.cos(br)) * np.sin(ar)
    z = c * np.sin(br)
    
    return x, y, z

In [73]:
TransformedGALx = []
TransformedGALy = []
TransformedGALz = []

for i in range(len(GalRa)):
    
    CordsRa = list(map(int, GalRa[i].split()))
    CordsDec = list(map(int, GalDec[i].split()))
    
    d = RaDec_to_Cartesian_GAL(CordsRa[0], CordsRa[1], CordsRa[2], CordsDec[0], CordsDec[1], CordsDec[2])
    
    TransformedGALx.append(d[0])
    TransformedGALy.append(d[1])
    TransformedGALz.append(d[2])

In [74]:
potentialMatchesX = []
potentialMatchesY = []
potentialMatchesZ = []
potentialMatchesIndex = []

matchingFRB_X = []
matchingFRB_Y = []
matchingFRB_Z = []
matchingFRB_Index = []

In [59]:
def FRB_Galaxy_Match(Min_Dist):
    
    for i in range(len(FRBra)):
        
        for j in range(len(GalRa)):
            
            Distance = np.sqrt((TransformedFRBx[i] - TransformedGALx[j])** 2 + (TransformedFRBy[i] - TransformedGALy[j])** 2 + (TransformedFRBz[i] - TransformedGALz[j])** 2)
            
            if Distance <= Min_Dist:
                
                potentialMatchesX.append(TransformedGALx[j])
                potentialMatchesY.append(TransformedGALy[j])
                potentialMatchesZ.append(TransformedGALz[j])
                potentialMatchesIndex.append(j)
                
                matchingFRB_X.append(TransformedFRBx[i])
                matchingFRB_Y.append(TransformedFRBy[i])
                matchingFRB_Z.append(TransformedFRBz[i])
                matchingFRB_Index.append(i)

In [60]:
FRB_Galaxy_Match(0.01)

In [61]:
print("List of Galaxies close to an FRB (x,y,z)")
print(potentialMatchesX)
print(potentialMatchesY)
print(potentialMatchesZ)
print("")
print("List of corresponding FRBs (x,y,z)")
print(matchingFRB_X)
print(matchingFRB_Y)
print(matchingFRB_Z)
print("")
print("Number of galaxies:")
print(len(potentialMatchesX))
print("")
print("Number of FRBs")
print(len(matchingFRB_X))
print("")
print("Galaxy Index")
print(potentialMatchesIndex)
print("")
print("FRB Index")
print(matchingFRB_Index)
print("")


List of Galaxies close to an FRB (x,y,z)
[-0.7197710605328761, -0.5374073034320644, 0.8434301480396713, 0.8434301480396713]
[0.3048455096966673, 0.5034839446570972, -0.4464950619926327, -0.4464950619926327]
[0.623697711745969, 0.6765333012427445, 0.2987770824443754, 0.2987770824443754]

List of corresponding FRBs (x,y,z)
[-0.717367228593971, -0.5383592356336717, 0.8386844025124999, 0.8386844025124999]
[0.3015538872813237, 0.4985266767900566, -0.45158562767524246, -0.45158562767524246]
[0.6280521573921454, 0.6794413042615168, 0.3044320841491716, 0.3044320841491716]

Number of galaxies:
4

Number of FRBs
4

Galaxy Index
[272, 36, 172, 172]

FRB Index
[122, 195, 245, 591]



In [62]:
'''
FRBbdm = df['bonsai_dm'].tolist()
FRBdmfit = df['dm_fitb'].tolist()
FRBdmfit_er = df['dm_fitb_err'].tolist()
'''

def Print_Info (GalIndex, FRBindex): 
    
    print("Lens Galaxy Pair " + str(GalIndex))
    print("Name: ")
    print(GalName[GalIndex])
    print("RA: ")
    print(NewGalRa[GalIndex])
    print("DEC: ")
    print(NewGalDec[GalIndex])
    print("Redshift: ")
    print(GalZ[GalIndex])
    #print("f_z: ")
    #print(Galf_z[GalIndex])
    print("")

    print("Matching FRB " + str(FRBindex))
    print("Name: ")
    print(FRBname[FRBindex])
    print("RA: ")
    print(FRBra[FRBindex])
    print("DEC: ")
    print(FRBdec[FRBindex])
    print("Redshift: ")
    print(FRBz[FRBindex])
    print("Bonsai DM: ")
    print(FRBbdm[FRBindex])
    print("DM using fitting algorithm fitburst: ")
    print(FRBdmfit[FRBindex])
    print("Fitting Algorithm DM error")
    print(FRBdmfit_er[FRBindex])
    print("SNR (Signal to Noise Ratio): ")
    print(FRBbsnr[FRBindex])
    print("")
    print("---------------------------------")
    print("")
    


In [63]:
Print_Info(272, 122)
Print_Info(36, 195)
Print_Info(172, 245)
Print_Info(172, 591)
    

Lens Galaxy Pair 272
Name: 
PS1J1028+3820
RA: 
157.045826
DEC: 
37.66
Redshift: 
0.35

Matching FRB 122
Name: 
FRB20181128B
RA: 
157.2
DEC: 
38.28
Redshift: 
0.4561418783798722
Bonsai DM: 
454.5
DM using fitting algorithm fitburst: 
456.55
Fitting Algorithm DM error
0.03
SNR (Signal to Noise Ratio): 
13.3

---------------------------------

Lens Galaxy Pair 36
Name: 
PS1J0907+4233
RA: 
136.866648
DEC: 
41.449444444444445
Redshift: 
0.495

Matching FRB 195
Name: 
FRB20181229A
RA: 
137.2
DEC: 
42.0
Redshift: 
0.8993248102206026
Bonsai DM: 
955.9
DM using fitting algorithm fitburst: 
955.574
Fitting Algorithm DM error
0.004
SNR (Signal to Noise Ratio): 
23.6

---------------------------------

Lens Galaxy Pair 172
Name: 
PS1J2208+1712
RA: 
332.10415
DEC: 
16.789166666666667
Redshift: 
0.4682

Matching FRB 245
Name: 
FRB20190117A
RA: 
331.7
DEC: 
17.4
Redshift: 
0.3789528597945124
Bonsai DM: 
393.1
DM using fitting algorithm fitburst: 
393.36
Fitting Algorithm DM error
0.002
SNR (Signal to